In [1]:
import json
import numpy
import operator

In [2]:
ls ../../data/pure_agg_gt/saved_dictionaries/

norm_pure_agg_gt_values.json     pure_agg_gt_quest2ads.json
pure_agg_gt_answer_key.json      pure_agg_metric_parameters.json


In [3]:
def clean_sub(sub, index_of_ad, index_of_score, expected_length):
    seen = []
    uniqs = []
    # explicitly check ordering
    i = 100
    sub.sort(key=operator.itemgetter(index_of_score), reverse=True)
    for entry in sub:
        # skip over aggregate single answer
        # this method made more complicated due
        # to inconsistent submission format
        if len(entry) == expected_length:
            if entry[index_of_score] > i:
                print "ORDERING PROBLEM"
                return None
            i = entry[index_of_score]
        #else:
        #    print type(entry)
    # de-dupe
    for entry in sub:
        # skip over aggregate single answer
        # this method made more complicated due
        # to inconsistent submission format
        if len(entry) == expected_length:
            if entry[index_of_ad] not in seen:
                uniqs.append(entry)
                seen.append(entry[index_of_ad])
    # explicitly sort by score after de-duping
    uniqs.sort(key=operator.itemgetter(index_of_score), reverse=True)
    # explicitly check ordering
    i = 100
    for entry in uniqs:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
    return uniqs

In [4]:
def AvgPrecision(gt, sub, index_of_ad, index_of_score):
    score = 100
    seen = []
    rel_docs = len(gt)
    num_hits = 0
    num_retrieved = 0
    cumpre = 0
    for entry in sub:
        num_retrieved += 1
        ad_id = entry[index_of_ad]
        # excessive checking of dupes
        if ad_id in seen:
            print "DUPLICATE"
            return None
        seen.append(ad_id)
        # excessive checking of order
        if entry[index_of_score] > score:
            print "ORDER PROBLEM"
            return None
        score = entry[index_of_score]
        if ad_id in gt:
            num_hits += 1
            cumpre += num_hits/float(num_retrieved)
            
    return cumpre/rel_docs

In [5]:
def AggMetric(gt_answ, sub_answ, feature):
    function = pure_agg_metric_params[feature]['function']
    amount = pure_agg_metric_params[feature]['amount']
    
    if feature in ['price', 'height', 'weight', 'age']:
        sub_answ = float(sub_answ)
        
        # Allow for centimeter height
        if feature == 'height':
            if sub_answ > 100:
                sub_answ *= 0.393701
                
        # Allow for kilogram weight
        if feature == 'weight':
            if sub_answ < 90:
                sub_answ *= 2.20462
                
        diff = gt_answ - sub_answ
        if function == 'multiply':
            if numpy.absolute(diff) <= (gt_answ * amount):
                metric = 1
            else:
                metric = 0
        elif function == 'add':
            if numpy.absolute(diff) <= amount:
                metric = 1
            else:
                metric = 0
    if function == 'match':
        if gt_answ in sub_answ.lower():
            metric = 1
        else:
            metric = 0
            
    return metric

In [6]:
pure_agg_submission_files = {}
pure_agg_submission_files['Georgetown'] = '../../data/team_submissions/Georgetown/GroundTruth/pure_agg_answer.json'
pure_agg_submission_files['Uncharted'] = '../../data/team_submissions/Uncharted/GroundTruth/uncharted_GT_pure_aggregate_answers_JSON_LINES.json'
pure_agg_submission_files['ISI'] = '../../data/team_submissions/ISI/GroundTruth/parsed_pure_aggregate_gt_v3_all_answers.jl'

gt_file_path = '../../data/pure_agg_gt/saved_dictionaries/'
gt_docs_file = gt_file_path + 'pure_agg_gt_quest2ads.json'
gt_agg_file = gt_file_path + 'pure_agg_gt_answer_key.json'

agg_parameters_file = gt_file_path + 'pure_agg_metric_parameters.json'

## Load Submissions

In [7]:
Submissions = {}

for team in pure_agg_submission_files.keys():
    pf_sub_file = pure_agg_submission_files[team]
    subs = []
    with open(pf_sub_file, 'r') as f:
        for line in f:
            temp = json.loads(line)
            subs.append(temp)
    Submissions[team] = subs
    del subs

## Load GT

In [8]:
f = open(gt_docs_file, 'r')
pure_agg_gt = eval(f.read())
f.close()

f = open(gt_agg_file, 'r')
pure_agg_answ_gt = eval(f.read())
f.close()

f = open(agg_parameters_file, 'r')
pure_agg_metric_params = eval(f.read())
f.close()

In [ ]:
NewEval = {}

## Create Dictionary

In [ ]:
print 'When checking manually, note the implicit unit conversions performed in AggMetric()'

for team in Submissions.keys():

    NewEval['Pure Aggregate'] = {}
    
    sub_data = Submissions[team]
    gt_data = pure_agg_gt

    try:
        del submissions
    except:
        pass
    for sub in sub_data:
        submissions = clean_sub(sub['answers'], 1, 2, 3)

        q_id = sub['question_id']
        gt = gt_data[q_id]
        NewEval['Pure Aggregate'][q_id] = {}
        NewEval['Pure Aggregate'][q_id]['avg_precision'] = AvgPrecision(gt, submissions, 1, 2)
        
        # Calculate Aggregate Metric
        if len(sub['aggregate']) == 1:
            metric = AggMetric(pure_agg_answ_gt[q_id]['agg_answer'], sub['aggregate'][0], pure_agg_answ_gt[q_id]['feature'])
        elif len(sub['aggregate']) > 1:
            print "MULTIPLE AGG SUBMISSIONS"
        elif len(sub['aggregate']) == 0:
            metric = 0
        NewEval['Pure Aggregate'][q_id]['agg_metric'] = metric
    
    # Manually check matching
    print ' '
    print team.upper()
    print ' '  
    for entry in Submissions[team]:
        q_id = entry['question_id']
        sub_agg = entry['aggregate']
        gt_agg = pure_agg_answ_gt[q_id]['agg_answer']
        points = NewEval['Pure Aggregate'][q_id]['agg_metric']
        print '{0} {1} {2}'.format(sub_agg, gt_agg, points)
        
    output_file = team + '/' + team + '_pure_agg_gt_results_rewrite.json'
    with open(output_file, 'w') as outfile:
        json.dump(NewEval, outfile, indent=2)

## Save Evaluation Dict

In [ ]:
save_file = extractions +'.json'
with open(save_file, 'w') as outfile:
    json.dump(NewEval, outfile, indent=2)

In [ ]:
ls 

In [ ]:
news = 3 
news *= 2
news

In [9]:
pure_agg_gt

{'1': ['E45BD1272392D722CB062DBFFF632B65D84CD7AA5E1504CD3F22A4EBB3F546C3',
  'B918710E96782403D4E05D7FDDB7FF325DB26970B4950FFAA2A43FC98E0A8BD0',
  '15404EAC0AEC9BB3D77C0BEBBCC16D080A0987D1B295EA1704E153902419F728',
  'FB3E84BBF007FB574E50CF2403CB110FB789A452BD6726327EA86968252EC683',
  'FBDA87DBCBACECE04042C36A5A657286EB3793136EA8EDD9398C39A1DF980D25',
  '96BE675A79D7601EE999A1EBD7CF1AC7CAF7C8429EFEF444F1371BD541978A18',
  '099CF86376D89693C4E14C466E899CFC0857E755FE57C049B94ACDDAD1720A0B',
  '631637F1566B35AB4700264E2146559548170AC4BC05EED5D74005361EC8512E',
  '924C066F4585D3FCD2757A1BA7D38AFB089AD69953CD0A4E75AC80667F0C6F6B',
  'B434771FCA260ABA0C983A740DC72279B5CC539CDDA1AEFBF66B2BE674DF5F17',
  '10AC14AC64077F43076A9D35B586D4F886987564E9079A59DE6D2F38602E4690',
  'F6BB9DA7EC4D9CDE6663B010E0B86FCA8CE7D0EAA5EB5E66F05E65A9F21CD5F3',
  'C1139EDE4D292E1E2E440219BE4087C313E6F8BDCBD9D2255A5332F70838E6E1',
  '7389E5C7985EC8D639B62132814E4C0C895EBC36D4D4533C1A5C3D8F85381C27',
  'A43A49D22336